In [1]:
import os

In [2]:
%pwd

'e:\\Chicken_Disease_classifier\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\Chicken_Disease_classifier'

In [5]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class Model_train:
    root_dir:Path
    trained_model_path:Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


    

In [8]:
from CHICKEN_DISEASE_CLASSIFIER.constants import *
from CHICKEN_DISEASE_CLASSIFIER.utils.common import read_yaml,create_directories
from CHICKEN_DISEASE_CLASSIFIER.entity.config_entity import  CallBacks

In [10]:
class ConfugurationManager:
    def __init__(self,
                 config_filepath=config_file_path,
                 param_filepath=param_file_path):
        
        self.config=read_yaml(config_filepath)
        self.param=read_yaml(param_filepath)

        create_directories([self.config.artifacts_root])

    def get_call_backs(self)->CallBacks:
        config=self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(config.tensorboard_root_log_dir),Path(model_ckpt_dir)])

        prepare_callback=CallBacks(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback
    

    def get_training_model(self)->Model_train:
        training=self.config.training
        prepare_base_model=self.config.Prepare_base_model
        params=self.param
        training_data_dir=os.path.join(self.config.Data_ingestion.root_dir)

        create_directories([training.root_dir])

        training_config=Model_train(
                root_dir=Path(training.root_dir),
                trained_model_path=Path(training.trained_model_path),
                updated_base_model_path=prepare_base_model.updated_base_model_dir,
                training_data= Path(training_data_dir),
                params_epochs=params.EPOCHS,
                params_batch_size= params.BATCH_SIZE,
                params_is_augmentation= params.AUGMENTATION,
                params_image_size= params.IMAGE_SIZE)
        
        return training_config
    


In [14]:
import time
import tensorflow as tf


In [14]:
class PrepareCallback:
    def __init__(self, config: CallBacks):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
    

In [15]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import tensorflow as tf

In [16]:

class Model_trainer:
    def __init__(self,config:Model_train):
        self.config=config

    def base_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    def train_valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.20
            )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear" )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs)

        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs)
        else:
            train_datagenerator = valid_datagenerator

        
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



        
    
    

In [17]:
try:
    config=ConfugurationManager()
    get_callbacks_config=config.get_call_backs()
    get_callbacks=PrepareCallback(config=get_callbacks_config)
    callback_list=get_callbacks.get_tb_ckpt_callbacks()


 
    training_config=config.get_training_model()

    train_data=Model_trainer(config=training_config)

    train_data.base_model()
    train_data.train_valid_generator()
    train_data.train(callback_list=callback_list)

except Exception as e:
    raise e



    

[2023-08-15 21:53:44,629: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-15 21:53:44,637: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-15 21:53:44,637: INFO: common: created directory at: artifacts]
[2023-08-15 21:53:44,645: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2023-08-15 21:53:44,645: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-08-15 21:53:44,653: INFO: common: created directory at: artifacts/training]


Found 1612 images belonging to 4 classes.
Found 6455 images belonging to 4 classes.
 32/403 [=>............................] - ETA: 1:11:58 - loss: 17.9214 - accuracy: 0.3379

In [5]:
from tensorboard import program

# Specify the directory where your TensorBoard logs are located
log_dir = 'artifacts/prepare_callbacks/tensorboard_log_dir'

# Initialize TensorBoard's default program
tb = program.TensorBoard()

# Launch TensorBoard with the specified log directory
tb.configure(argv=[None, '--logdir', log_dir])
url = tb.launch()
print(f'TensorBoard is now available at {url}')

TensorBoard is now available at http://localhost:6006/
